# Aprendizado Supervisionado

## Exercício 04 - Testando todos os Modelos

Neste exemplo, vamos testar todos os modelso estudados no curso. Vamo aplica-los a classificação de textos. A base para esse exemplo foi gerada na atividade de clusterização de documentos. Iremos utiliza-la como base de treinamento. A proposta é usar os 100 top filmes para classificar outros 100 top filmes do IMDb. Esse tipo de abordagem pode ser utilizada para realizar uma classificação inicial quando temos muitos dados e não conseguimos classifica-los manualmente. Claro que para a classificação ser mais efetiva precisaríamos realizar diversos testes na criação dos clusters e validarmos a clusterização realizada. 

### Dataset

O primeiro passo é carregar a base de dados. Ela é composta de 100 filmes e para cada filme temos: 

* **imdbid:** Identificador no IMDb
* **title:** Título do Filme
* **synopse:** Sinópse do Filme
* **class:** classificação do filme

In [18]:
# Imports Necessários
import nltk
import re
import pandas as pd
from sklearn.cluster import KMeans
from imdbpie import Imdb
from nltk.stem.snowball import SnowballStemmer
from sklearn.externals import joblib

In [15]:
dataset = joblib.load("../UnsupervisedLearning/moviesclassifieds.pkl")
data = pd.DataFrame(dataset)
data = data.set_index(['imdbid'])
data.head()

,class,sinopsis,title
imdbid,,,
tt0111161,2,Chronicles the experiences of a formerly succe...,The Shawshank Redemption
tt0068646,1,When the aging head of a famous crime family d...,The Godfather
tt0071562,0,The continuing saga of the Corleone crime fami...,The Godfather: Part II
tt0468569,0,Set within a year after the events of Batman B...,The Dark Knight
tt0050083,2,The defense and the prosecution have rested an...,12 Angry Men


Vamos carregar também o identificador das classes geradas:

In [16]:
dict_class = joblib.load("../UnsupervisedLearning/moviesclass.pkl")
dict_class

{0: [b'new', b'years', b'crime', b'friends', b'forcing', b'time'],
 1: [b'powerful', b'war', b'world', b'wants', b'forcing', b'man'],
 2: [b'murder', b'man', b'death', b'believes', b'help', b'time'],
 3: [b'kill', b'men', b'plan', b'crime', b'young', b'help'],
 4: [b'life', b'love', b'meet', b'live', b'man', b'working']}

O passo seguinte é gerar nosso **X** e **y** para os algoritmos de classificação. Vamos utilizar a mesma abordagem para a criação da matriz tf-idf no exercício de clusterização. Sendo assim, irei replicar alguns métodos. Detalhes de sua explicação estão no exercício 01 de Algoritmos Não-Supervisionados. 

In [19]:
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    # first tokenize by sentence, then by word to ensure that punctuation is caught as it's own token
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [29]:
data.head()

,class,sinopsis,title
imdbid,,,
tt0111161,2,Chronicles the experiences of a formerly succe...,The Shawshank Redemption
tt0068646,1,When the aging head of a famous crime family d...,The Godfather
tt0071562,0,The continuing saga of the Corleone crime fami...,The Godfather: Part II
tt0468569,0,Set within a year after the events of Batman B...,The Dark Knight
tt0050083,2,The defense and the prosecution have rested an...,12 Angry Men


In [32]:
list_of_synopses = data['sinopsis'].tolist()
classes = data['class'].tolist()

In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer

#define vectorizer parameters
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, max_features=200000,
                                 min_df=0.2, stop_words='english',
                                 use_idf=True, tokenizer=tokenize_and_stem, ngram_range=(1,3))

%time tfidf_matrix = tfidf_vectorizer.fit_transform(list_of_synopses) #fit the vectorizer to synopses

print(tfidf_matrix.shape)

CPU times: user 1.93 s, sys: 30.9 ms, total: 1.96 s
Wall time: 1.99 s
(100, 88)


In [34]:
X = tfidf_matrix
y = classes

## Atividade

01) Aplique os modelos estudados juntamente com o RandomForest e o NaiveByes na base de dados geradas. Escolha o melhor classificador. 

02) Utilize o melhor classificador para gerar classificar outros 100 textos do IMDb. Neste casso, utilize os top250 textos. Utilizamos 100 para treinar e podemos utilizar os textos de 101 a 200 para testar. 